# Stage 3: Replay Harnesses — Reproducibility & Rich Metrics

---

## 🎯 What You'll Learn

By the end of this walkthrough, you'll understand:

1. **What replay harnesses are** and why reproducibility matters
2. **How recording and replay works** behind the scenes
3. **The evaluation metrics** used in ML/AI — precision, recall, groundedness, faithfulness
4. **How to use LLM-as-judge** for generation quality assessment

---

## 📚 Understanding Replay Harnesses

### The Problem with Live Evaluation

In Stages 1 and 2, every test run calls the actual agent:
- LLM responses vary (even with temperature=0)
- External services may behave differently
- Tests aren't fully reproducible
- Each run costs money

**Replay harnesses solve this** by recording sessions and replaying them deterministically.

### What Are Replay Harnesses?

A replay harness has three components:

1. **Recorder** — Captures agent sessions (query, tool calls, responses)
2. **Player** — Replays sessions with cached responses (no LLM calls)
3. **Evaluator** — Computes rich ML metrics on replayed sessions

| Stage 1 & 2 | Stage 3: Replay |
|-------------|-----------------|
| Live agent calls | Cached responses |
| Variable results | Deterministic |
| Simple pass/fail | Rich metrics |
| $ per run | Free replay |

---

## 🏗️ Where Replay Harnesses Fit: The Eval Maturity Model

```
┌─────────────────────────────────────────────────────────────────┐
│                    EVAL FRAMEWORK MATURITY                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Stage 5: Experiments      ← Compare configurations            │
│      ▲                                                          │
│  Stage 4: Rubrics          ← Multi-dimensional scoring         │
│      ▲                                                          │
│  ★ Stage 3: REPLAY HARNESSES ★ ← Reproducibility (YOU ARE HERE)│
│      ▲                                                          │
│  Stage 2: Labeled Scenarios← Coverage mapping (completed)      │
│      ▲                                                          │
│  Stage 1: Golden Sets      ← Baseline correctness (completed)  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

Replay harnesses add **reproducibility** and **proper ML metrics** to your evaluation framework.

## 🔍 Behind the Scenes: The Record/Replay Cycle

Here's how the replay harness works:

```
┌──────────────────────────────────────────────────────────────────────┐
│                    RECORD/REPLAY CYCLE                                │
├──────────────────────────────────────────────────────────────────────┤
│                                                                       │
│  PHASE 1: RECORD (one-time, costs $)                                 │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │                                                             │     │
│  │  Query ──► Agent ──► Tool Calls ──► LLM Response            │     │
│  │                          │              │                   │     │
│  │                          ▼              ▼                   │     │
│  │                    ┌──────────────────────┐                 │     │
│  │                    │   Session JSON       │                 │     │
│  │                    │   - query            │                 │     │
│  │                    │   - tool_calls       │                 │     │
│  │                    │   - response         │                 │     │
│  │                    │   - sources          │                 │     │
│  │                    │   - annotations      │                 │     │
│  │                    └──────────────────────┘                 │     │
│  │                              │                              │     │
│  │                              ▼                              │     │
│  │                    fixtures/session-001.json                │     │
│  │                                                             │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                                                                       │
│  PHASE 2: ANNOTATE (human review)                                    │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │  Add ground truth:                                          │     │
│  │   • relevant_sources: ["refund_policy.md"]                  │     │
│  │   • expected_facts: ["30-day window", "8 refunds"]          │     │
│  │   • expected_tools: ["vector_search", "sql_query"]          │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                                                                       │
│  PHASE 3: REPLAY (unlimited, free)                                   │
│  ┌─────────────────────────────────────────────────────────────┐     │
│  │                                                             │     │
│  │  Load Session ──► Compare Against Annotations               │     │
│  │                              │                              │     │
│  │                              ▼                              │     │
│  │                    ┌──────────────────────┐                 │     │
│  │                    │   Compute Metrics    │                 │     │
│  │                    │   - Precision: 1.0   │                 │     │
│  │                    │   - Recall: 1.0      │                 │     │
│  │                    │   - Groundedness: 95%│                 │     │
│  │                    │   - Faithfulness: 100│                 │     │
│  │                    └──────────────────────┘                 │     │
│  │                                                             │     │
│  └─────────────────────────────────────────────────────────────┘     │
│                                                                       │
└──────────────────────────────────────────────────────────────────────┘
```

**Key insight:** You record once, then replay and evaluate unlimited times for free.

## 📊 The Evaluation Metrics

Replay harnesses use **proper ML/AI metrics** instead of just pass/fail. Let's understand each category:

### Retrieval Metrics (RAG Quality)

These measure how well your agent finds relevant documents:

| Metric | Formula | What It Tells You |
|--------|---------|-------------------|
| **Precision** | relevant_retrieved / total_retrieved | "Of what we found, how much was useful?" |
| **Recall** | relevant_retrieved / total_relevant | "Of what exists, how much did we find?" |
| **F1 Score** | 2 × (P × R) / (P + R) | Harmonic mean of precision and recall |
| **MRR** | 1 / rank_of_first_relevant | "How quickly do we find something good?" |

**Example:**
- Ground truth relevant docs: `[doc_A, doc_B]`
- Agent retrieved: `[doc_A, doc_C]`
- Precision = 1/2 = 50% (only doc_A was relevant)
- Recall = 1/2 = 50% (missed doc_B)

### Generation Metrics (LLM-as-Judge)

These measure response quality using another LLM as an evaluator:

| Metric | What It Measures | How It Works |
|--------|-----------------|--------------|
| **Groundedness** | Claims supported by sources | Judge checks each claim against retrieved docs |
| **Faithfulness** | No hallucinated facts | Judge looks for made-up information |
| **Relevance** | Answers the question (1-5) | Judge rates if response addresses the query |
| **Completeness** | Covers all aspects | Judge checks if all parts are addressed |

### Tool Metrics

| Metric | What It Measures |
|--------|-----------------|
| **Tool Accuracy** | Did it use the correct tools? |
| **Tool Efficiency** | Did it avoid unnecessary calls? |

In [ ]:
# Setup: Import required modules
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent / "setup_agent"))

from recorder import Session, load_session, list_sessions, record_session
from player import replay_session, get_session_summary
from metrics import precision, recall, f1_score, mrr, groundedness, faithfulness, relevance
from evaluator import evaluate_session

## 💻 Hands-On: Working with Sessions

Let's explore the recorded sessions (fixtures) and see how replay works.

In [ ]:
# List existing recorded sessions
sessions = list_sessions()
print(f"📁 Found {len(sessions)} recorded sessions:\n")

for session_id in sessions:
    try:
        summary = get_session_summary(session_id)
        print(f"  📝 {session_id}")
        print(f"     Query: {summary['query'][:50]}...")
        print(f"     Tools: {summary['tool_calls']}")
        print(f"     Has annotations: {summary['has_annotations']}")
        print()
    except Exception as e:
        print(f"  ⚠️ {session_id}: Could not load - {e}")

## 🧮 Understanding Retrieval Metrics

Let's compute precision and recall manually to understand what they mean:

In [ ]:
# Example: Computing retrieval metrics
print("🧮 RETRIEVAL METRICS EXAMPLE")
print("=" * 50)

# Scenario: Agent needs to find refund-related docs
relevant_docs = ["refund_policy.md", "pricing_strategy.md"]  # Ground truth
retrieved_docs = ["refund_policy.md", "pto_policy.md"]        # What agent found

print(f"\nGround truth relevant: {relevant_docs}")
print(f"Agent retrieved:       {retrieved_docs}")

# Calculate metrics
p = precision(retrieved_docs, relevant_docs)
r = recall(retrieved_docs, relevant_docs)
f1 = f1_score(retrieved_docs, relevant_docs)
mrr_score = mrr(retrieved_docs, relevant_docs)

print(f"\n📊 Results:")
print(f"   Precision: {p:.2f} → Of 2 retrieved, 1 was relevant (50%)")
print(f"   Recall:    {r:.2f} → Of 2 relevant, 1 was found (50%)")
print(f"   F1 Score:  {f1:.2f} → Harmonic mean")
print(f"   MRR:       {mrr_score:.2f} → First relevant at position 1")

# Perfect retrieval example
print("\n" + "=" * 50)
print("Perfect retrieval scenario:")
retrieved_perfect = ["refund_policy.md", "pricing_strategy.md"]
print(f"Retrieved: {retrieved_perfect}")
print(f"Precision: {precision(retrieved_perfect, relevant_docs):.2f}")
print(f"Recall:    {recall(retrieved_perfect, relevant_docs):.2f}")

## 🤖 LLM-as-Judge: Generation Metrics

For generation quality, we use another LLM to evaluate the response. This is called "LLM-as-judge" and is the standard approach for evaluating language quality.

### How Groundedness Works

The judge receives the sources and response, then checks each claim:

```
Sources: "Refund policy: 30-day guarantee on annual plans..."
Response: "We offer a 30-day guarantee for annual plans."

Claim 1: "30-day guarantee" → GROUNDED (in source)
Claim 2: "annual plans" → GROUNDED (in source)

Score: 2/2 = 100% grounded
```

### How Faithfulness Differs

Faithfulness specifically checks for **hallucinations** — facts that are fabricated and NOT in sources.

In [ ]:
# Example: Evaluating response groundedness
print("🤖 LLM-AS-JUDGE EXAMPLE")
print("=" * 50)

# Simulated response and sources
sources = [
    "Refund Policy: We offer a 30-day money-back guarantee on all annual plans. "
    "Customers must submit refund requests through our support portal."
]

# Good response (grounded)
good_response = "Our refund policy provides a 30-day money-back guarantee for annual plans. To request a refund, contact support."

# Bad response (with hallucination)
bad_response = "Our refund policy provides a 30-day money-back guarantee for annual plans. We also offer a 60-day guarantee for enterprise customers."

print("Sources:", sources[0][:80] + "...")
print()
print("Good response (should be grounded):")
print(f"  '{good_response}'")
print()
print("Bad response (contains hallucination):")
print(f"  '{bad_response}'")
print("  ^ '60-day guarantee for enterprise' is NOT in the source!")

# Note: Actual groundedness check requires API call
print("\n⚠️ Note: Full groundedness evaluation requires an LLM call (costs $)")
print("   Run evaluator.py to see real scores")

## 🔴 Recording a New Session

Let's record a new session to see the full flow. This will:
1. Send a query to the actual agent (costs $)
2. Capture all tool calls and the response
3. Save it to a JSON fixture file

In [ ]:
# Record a new session (this calls the actual agent - costs $)
# Uncomment to run:

# session = record_session(
#     query="What is the home office equipment stipend?",
#     session_id="demo_session"
# )
# 
# print(f"\\n✅ Session recorded!")
# print(f"   ID: {session.session_id}")
# print(f"   Tools used: {[tc.tool_name for tc in session.tool_calls]}")
# print(f"   Sources found: {session.retrieved_sources}")

print("💡 To record a session, uncomment the code above and run the cell.")
print("   This will call the actual agent (costs $).")

## ▶️ Replaying a Session

Once recorded, you can replay sessions unlimited times without API calls:

In [ ]:
# Replay an existing session
sessions = list_sessions()

if sessions:
    session_id = sessions[0]  # Use first available session
    print(f"▶️ Replaying session: {session_id}")
    print("=" * 50)
    
    session = replay_session(session_id)
    
    print(f"\n📝 Query: {session.query}")
    print(f"🔧 Tools: {[tc.tool_name for tc in session.tool_calls]}")
    print(f"📚 Sources: {session.retrieved_sources}")
    print(f"\n📄 Response preview:")
    print("-" * 50)
    print(session.response[:400] + "..." if len(session.response) > 400 else session.response)
else:
    print("⚠️ No sessions found. Record one first using record_session()")

## 📊 Full Session Evaluation

The evaluator runs all metrics on a session:

In [ ]:
# Run full evaluation on a session
sessions = list_sessions()

if sessions:
    session_id = sessions[0]
    session = load_session(session_id)
    
    print(f"📊 Evaluating session: {session_id}")
    print("=" * 50)
    
    # This will call LLM judges for generation metrics
    results = evaluate_session(session, verbose=True)
    
    print(f"\n🏆 Overall Score: {results['overall']:.2f}")
else:
    print("⚠️ No sessions to evaluate. Record one first.")

## 🎓 Key Takeaways

1. **Record once, replay forever** — Deterministic evaluation without API costs
2. **Proper ML metrics** — Precision, recall, F1 for retrieval quality
3. **LLM-as-judge** — Groundedness and faithfulness for generation quality
4. **Annotations are key** — Ground truth enables meaningful metrics
5. **Version your fixtures** — Commit session JSONs to git for reproducibility

---

## 📈 Metrics Quick Reference

| Metric | Good Score | Warning Signs |
|--------|------------|---------------|
| Precision | > 0.8 | Retrieving irrelevant docs |
| Recall | > 0.8 | Missing relevant docs |
| Groundedness | > 0.9 | Claims not supported by sources |
| Faithfulness | 1.0 | Any hallucination is a problem |
| Relevance | 4-5/5 | Response doesn't address query |

---

## ⏭️ What's Next?

Replay harnesses give you rich metrics, but they're still pass/fail in nature. What about **gradual quality degradation**?

- A response can be technically correct but poorly written
- Some dimensions matter more than others (accuracy > clarity for compliance)
- You want to track quality trends over time

**Stage 4: Rubric-Based Evaluation** addresses this with multi-dimensional scoring:

```bash
cd ../stage_4_rubrics
uv run jupyter notebook walkthrough.ipynb
```